In [92]:
## Import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, AutoDateFormatter, AutoDateLocator, WeekdayLocator, MonthLocator, DayLocator, DateLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator
import numpy as np
import datetime, calendar
from datetime import timedelta
import matplotlib.patches as mpatches
%matplotlib tk

data = pd.read_pickle('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/entwuerfe/xls_testruns/doe_pickle_1458.pkl') # this one for average times

In [93]:
print(data.columns)
print(data.index)

Index(['tm', 'dt', 'yy', 'mm', 'ww', 'wd', 'dd', 'xl', 'hh', 'an', 'vb', 'vl',
       'ht', 'tt', 'acw', 'bz', 'dectt', 'decht', 'decacw'],
      dtype='object')
DatetimeIndex(['2016-11-14 00:00:00', '2016-11-14 01:00:00',
               '2016-11-14 02:00:00', '2016-11-14 03:00:00',
               '2016-11-14 04:00:00', '2016-11-14 05:00:00',
               '2016-11-14 06:00:00', '2016-11-14 07:00:00',
               '2016-11-14 08:00:00', '2016-11-14 09:00:00',
               ...
               '2017-10-08 21:30:00', '2017-10-08 21:45:00',
               '2017-10-08 22:00:00', '2017-10-08 22:15:00',
               '2017-10-08 22:30:00', '2017-10-08 22:45:00',
               '2017-10-08 23:00:00', '2017-10-08 23:15:00',
               '2017-10-08 23:30:00', '2017-10-08 23:45:00'],
              dtype='datetime64[ns]', length=23876, freq=None)


In [94]:
# functions that will be apllied on grouping, left out dt bz and hh; dt and bz should be indices, hh can be dropped
colfunx_collapse_dt={'dt':'first','yy':'first', 'mm':'first', 'ww':'first', 'wd':'first', 'dd':'first', 'xl':'first', 'an':'sum', 'vb':'sum', 'vl':'sum', 'ht':'sum', 'tt':'sum', 'acw':'sum'}
# group data by day and bearbzeit
data_grouped=data.groupby(['xl','bz']).agg(colfunx_collapse_dt)
# order columns for readability
colorder=['dt','yy','mm','dd','wd','ww','an','vb','vl','ht','tt','acw']
data_grouped=data_grouped[colorder]

In [95]:
data_grouped
print(data_grouped.index.levels)
com_index=pd.MultiIndex.from_product(data_grouped.index.levels) #  see here https://stackoverflow.com/questions/25656365/multiindex-from-product-of-unique-values-of-two-indexes
dframe_prod=pd.DataFrame(index=com_index)
dframe_prod
completeframe=pd.concat([data_grouped,dframe_prod], axis=1)

[[42688.0, 42689.0, 42690.0, 42691.0, 42692.0, 42693.0, 42694.0, 42695.0, 42696.0, 42697.0, 42698.0, 42699.0, 42700.0, 42701.0, 42702.0, 42703.0, 42704.0, 42705.0, 42706.0, 42707.0, 42708.0, 42709.0, 42710.0, 42711.0, 42712.0, 42713.0, 42714.0, 42715.0, 42716.0, 42717.0, 42718.0, 42719.0, 42720.0, 42721.0, 42722.0, 42723.0, 42724.0, 42725.0, 42726.0, 42727.0, 42728.0, 42729.0, 42730.0, 42731.0, 42732.0, 42733.0, 42734.0, 42735.0, 42736.0, 42737.0, 42738.0, 42739.0, 42740.0, 42741.0, 42742.0, 42743.0, 42744.0, 42745.0, 42746.0, 42747.0, 42748.0, 42749.0, 42750.0, 42751.0, 42752.0, 42753.0, 42754.0, 42755.0, 42756.0, 42757.0, 42758.0, 42759.0, 42760.0, 42761.0, 42762.0, 42763.0, 42764.0, 42765.0, 42766.0, 42767.0, 42768.0, 42769.0, 42770.0, 42771.0, 42772.0, 42773.0, 42774.0, 42775.0, 42776.0, 42777.0, 42778.0, 42779.0, 42780.0, 42781.0, 42782.0, 42783.0, 42784.0, 42785.0, 42786.0, 42787.0, ...], ['k', 'n']]


In [96]:
completeframe

dt      yy    mm    dd   wd    ww     an     vb     vl  \
42688.0 k  2016-11-14  2016.0  11.0  14.0  Mon  46.0  321.0  221.0  100.0   
        n         NaN     NaN   NaN   NaN  NaN   NaN    NaN    NaN    NaN   
42689.0 k  2016-11-15  2016.0  11.0  15.0  Tue  46.0  152.0  132.0   20.0   
        n         NaN     NaN   NaN   NaN  NaN   NaN    NaN    NaN    NaN   
42690.0 k  2016-11-16  2016.0  11.0  16.0  Wed  46.0   75.0   64.0   11.0   
        n         NaN     NaN   NaN   NaN  NaN   NaN    NaN    NaN    NaN   
42691.0 k  2016-11-17  2016.0  11.0  17.0  Thu  46.0   91.0   84.0    7.0   
        n         NaN     NaN   NaN   NaN  NaN   NaN    NaN    NaN    NaN   
42692.0 k  2016-11-18  2016.0  11.0  18.0  Fri  46.0  112.0   90.0   22.0   
        n         NaN     NaN   NaN   NaN  NaN   NaN    NaN    NaN    NaN   
42693.0 k  2016-11-19  2016.0  11.0  19.0  Sat  46.0   36.0   27.0    9.0   
        n         NaN     NaN   NaN   NaN  NaN   NaN    NaN    NaN    NaN   
42694.0 k  2016-11-20  2016.0  11.0  20.0  Sun  46.0    5.0    5.0    0.0   
        n         NaN     NaN   NaN   NaN  NaN   NaN    NaN    NaN    NaN   
42695.0 k  2016-11-21  2016.0  11.0  21.0  Mon  47.0  184.0  159.0   25.0   
        n         NaN     NaN   NaN   NaN  NaN   NaN    NaN    NaN    NaN   
42696.0 k  2016-11-22  2016.0  11.0  22.0  Tue  47.0   79.0   64.0   15.0   
        n         NaN     NaN   NaN   NaN  NaN   NaN    NaN    NaN    NaN   
42697.0 k  2016-11-23  2016.0  11.0  23.0  Wed  47.0   64.0   56.0    8.0   
        n         NaN     NaN   NaN   NaN  NaN   NaN    NaN    NaN    NaN   
42698.0 k  2016-11-24  2016.0  11.0  24.0  Thu  47.0   74.0   64.0   10.0   
        n         NaN     NaN   NaN   NaN  NaN   NaN    NaN    NaN    NaN   
42699.0 k  2016-11-25  2016.0  11.0  25.0  Fri  47.0   62.0   52.0   10.0   
        n         NaN     NaN   NaN   NaN  NaN   NaN    NaN    NaN    NaN   
42700.0 k  2016-11-26  2016.0  11.0  26.0  Sat  47.0   36.0   32.0    4.0   
        n         NaN     NaN   NaN   NaN  NaN   NaN    NaN    NaN    NaN   
42701.0 k  2016-11-27  2016.0  11.0  27.0  Sun  47.0    4.0    2.0    2.0   
        n         NaN     NaN   NaN   NaN  NaN   NaN    NaN    NaN    NaN   
42702.0 k  2016-11-28  2016.0  11.0  28.0  Mon  48.0   98.0   95.0    3.0   
        n         NaN     NaN   NaN   NaN  NaN   NaN    NaN    NaN    NaN   
...               ...     ...   ...   ...  ...   ...    ...    ...    ...   
43002.0 k         NaN     NaN   NaN   NaN  NaN   NaN    NaN    NaN    NaN   
        n  2017-09-24  2017.0   9.0  24.0  Sun  38.0    3.0    3.0    0.0   
43003.0 k  2017-09-25  2017.0   9.0  25.0  Mon  39.0  324.0  305.0   19.0   
        n  2017-09-25  2017.0   9.0  25.0  Mon  39.0    2.0    1.0    1.0   
43004.0 k  2017-09-26  2017.0   9.0  26.0  Tue  39.0  458.0  435.0   23.0   
        n  2017-09-26  2017.0   9.0  26.0  Tue  39.0    7.0    7.0    0.0   
43005.0 k  2017-09-27  2017.0   9.0  27.0  Wed  39.0  441.0  412.0   29.0   
        n  2017-09-27  2017.0   9.0  27.0  Wed  39.0    5.0    5.0    0.0   
43006.0 k  2017-09-28  2017.0   9.0  28.0  Thu  39.0  310.0  282.0   28.0   
        n  2017-09-28  2017.0   9.0  28.0  Thu  39.0    6.0    6.0    0.0   
43007.0 k  2017-09-29  2017.0   9.0  29.0  Fri  39.0  214.0  207.0    7.0   
        n  2017-09-29  2017.0   9.0  29.0  Fri  39.0    5.0    4.0    1.0   
43008.0 k  2017-09-30  2017.0   9.0  30.0  Sat  39.0   42.0   31.0   11.0   
        n  2017-09-30  2017.0   9.0  30.0  Sat  39.0   10.0    9.0    1.0   
43009.0 k         NaN     NaN   NaN   NaN  NaN   NaN    NaN    NaN    NaN   
        n  2017-10-01  2017.0  10.0   1.0  Sun  39.0    6.0    2.0    4.0   
43010.0 k  2017-10-02  2017.0  10.0   2.0  Mon  40.0  153.0  149.0    4.0   
        n  2017-10-02  2017.0  10.0   2.0  Mon  40.0    9.0    9.0    0.0   
43011.0 k  2017-10-03  2017.0  10.0   3.0  Tue  40.0    2.0    2.0    0.0   
        n  2017-10-03  2017.0  10.0   3.0  Tue  40.0    0.0    0.0    0.0   
43012.0 k  2017

In [97]:
#kernzeit=dg2.xs('k', level=1, drop_level=False)
#nebnzeit=dg2.xs('n', level=1, drop_level=False)
# both frames share the same index (level 0, i.e. datetime.date)

In [98]:
def get_year(dtobject):
    jahr=dtobject.year
    return jahr

In [99]:
def get_week(dtobject):
    yr=dtobject.year
    mn=dtobject.month
    wk=dtobject.isocalendar()[1]
    return yr,mn,wk


In [100]:
data_grouped3=data.groupby(['dt','bz']).agg({'tt':'sum','yy':'first','mm':'first','wd':'first','ww':'first','ht':'sum'})

#com_index2=pd.MultiIndex.from_product(data_grouped3.index.levels) #  see here https://stackoverflow.com/questions/25656365/multiindex-from-product-of-unique-values-of-two-indexes
#print(data_grouped3.index.levels)
#com_index2.levels

#data_grouped3.index.levels[1]==com_index2.levels[1]

In [122]:
sampleframe=data_grouped3.loc[[datetime.date(2016,12,7),datetime.date(2017,5,11),datetime.date(2017,5,14)],:].copy()
sampleframe=sampleframe[['yy','mm','ww','wd','ht','tt']]
print(sampleframe.describe)
sampleframe.reset_index(drop=True)
sampleframe


<bound method DataFrame.describe of                  yy  mm  ww   wd        ht        tt
dt         bz                                       
2016-12-07 k   2016  12  49  Wed  0.341759  0.183715
2017-05-11 k   2017   5  19  Thu  0.153368  0.137234
           n   2017   5  19  Thu  0.006157  0.003981
2017-05-14 n   2017   5  19  Sun  0.000000  0.000000>


yy  mm  ww   wd        ht        tt
dt         bz                                       
2016-12-07 k   2016  12  49  Wed  0.341759  0.183715
2017-05-11 k   2017   5  19  Thu  0.153368  0.137234
           n   2017   5  19  Thu  0.006157  0.003981
2017-05-14 n   2017   5  19  Sun  0.000000  0.000000

In [123]:
#prod_index=pd.MultiIndex.from_product(sampleframe.index.levels)

In [124]:
#df_prod=pd.DataFrame(index=prod_index)

In [125]:
sampleframe.index.levels

FrozenList([[2016-11-14, 2016-11-15, 2016-11-16, 2016-11-17, 2016-11-18, 2016-11-19, 2016-11-20, 2016-11-21, 2016-11-22, 2016-11-23, 2016-11-24, 2016-11-25, 2016-11-26, 2016-11-27, 2016-11-28, 2016-11-29, 2016-11-30, 2016-12-01, 2016-12-02, 2016-12-03, 2016-12-04, 2016-12-05, 2016-12-06, 2016-12-07, 2016-12-08, 2016-12-09, 2016-12-10, 2016-12-11, 2016-12-12, 2016-12-13, 2016-12-14, 2016-12-15, 2016-12-16, 2016-12-17, 2016-12-18, 2016-12-19, 2016-12-20, 2016-12-21, 2016-12-22, 2016-12-23, 2016-12-24, 2016-12-25, 2016-12-26, 2016-12-27, 2016-12-28, 2016-12-29, 2016-12-30, 2016-12-31, 2017-01-01, 2017-01-02, 2017-01-03, 2017-01-04, 2017-01-05, 2017-01-06, 2017-01-07, 2017-01-08, 2017-01-09, 2017-01-10, 2017-01-11, 2017-01-12, 2017-01-13, 2017-01-14, 2017-01-15, 2017-01-16, 2017-01-17, 2017-01-18, 2017-01-19, 2017-01-20, 2017-01-21, 2017-01-22, 2017-01-23, 2017-01-24, 2017-01-25, 2017-01-26, 2017-01-27, 2017-01-28, 2017-01-29, 2017-01-30, 2017-01-31, 2017-02-01, 2017-02-02, 2017-02-03, 201